Webscraping "Price", "Age", "Name" from LEGO website

In [1]:
#importing necessary libraries
!pip install selenium
!pip install webdriver_manager
!pip install openpyxl

import time
from getpass import getpass
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.support.ui import WebDriverWait

import os

import pathlib
import os
from os.path import join

from bs4 import BeautifulSoup
import re
import requests


In [111]:
#Creating the driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [112]:
#Getting into LEGO NL website
driver.get("https://www.lego.com/en-nl/categories/age")
time.sleep(2)

In [113]:
continue_icon = driver.find_element(By.XPATH, "/html/body/div[1]/div/div[4]/dialog/div/div[1]/div[1]/div/button")
continue_icon.click()
time.sleep(2)

In [114]:
secondary_icon = driver.find_element(By.CSS_SELECTOR, "button[kind = 'secondary']" )
secondary_icon.click()
time.sleep(2)

In [115]:


#Go through each Age icon
age_icons = driver.find_elements(By.CSS_SELECTOR, "a[class^='Linksstyles__Anchor-sc-684acv-0 friqQc CategoryLeafstyles__DetailsLink-is33yg-14 bzrWhW']")
age_group = {}
time.sleep(2)

In [116]:
age_links = [link.get_attribute("href") for link in age_icons] 

In [117]:

for age_link in age_links:
    time.sleep(5)
    # Open a new window
    driver.execute_script("window.open();")
    # Switch to the new window
    driver.switch_to.window(driver.window_handles[-1])
    # Open the link in the new window
    driver.get(age_link)
    time.sleep(1)
    # Wait for content to load
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, 'ProductLeaf_wrapper__H0TCb')))

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
        
    pic_list = []
    title_list = []
    age_list =[]
    price_list = []
    group_name = soup.find('h1').get_text(strip=True)

    #Check if the group_name is in the dictionary age_group, if not, add the new name
    if group_name not in age_group.keys():
        age_group[group_name] =[title_list, age_list, price_list, pic_list]

    # Find the div containing the pagination links
    pagination_div = soup.find('div', class_='Paginationstyles__PageLinks-npbsev-6 EncvI')

    # Find the last page number
    last_page = int(pagination_div.find_all('a')[-1].get_text(strip = True))

    # Use the href from the first page as the base for interation
    base_href = pagination_div.find('a')['href']
    
    # Loop through each page
    for i in range(last_page):
        if i == 0:
            page_link = 'https://www.lego.com'+base_href
        elif i <= (last_page-1):
            page_link = 'https://www.lego.com'+base_href + f"/?page={i+1}"
        else:
            continue

        time.sleep(1)
        driver.execute_script("window.open();")
        driver.switch_to.window(driver.window_handles[2])
        print("Trying to open:", page_link)
        driver.get(page_link)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        # Wait for content to load on the new page
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, 'ProductLeaf_wrapper__H0TCb')))

        #Find all the items on one page
        group_items= soup.find_all('article', attrs={'class': 'ProductLeaf_wrapper__H0TCb', 'data-test': 'product-leaf'})
        #Find all the details from each item on one page
        for item in group_items:
            picture = item.find('img')['src']
            pic_list.append(picture)
            title = item.find('h3', class_='ProductLeaf_titleRow__KqWbB').find('span', class_='markup').text
            title_list.append(title)
            age = item.find('span', class_='label-sm-medium ProductLeaf_attributeLabel__2VyjW').text
            age_list.append(age)
            try:
                price = item.find('div', class_='ProductLeaf_priceRow__RUx3P').text
            except AttributeError:
                price= 'None'

            price_list.append(price)
            time.sleep(1)
            
        driver.close()
        driver.switch_to.window(driver.window_handles[-1])
        i = i+1

    # Close the current window
    driver.close()
    # Switch back to the original window
    driver.switch_to.window(driver.window_handles[0])

# Close the main browser window
driver.quit()

age_group

Trying to open: https://www.lego.com/en-nl/categories/age-1-plus-years
Trying to open: https://www.lego.com/en-nl/categories/age-1-plus-years/?page=2
Trying to open: https://www.lego.com/en-nl/categories/age-1-plus-years/?page=3
Trying to open: https://www.lego.com/en-nl/categories/age-1-plus-years/?page=4


KeyboardInterrupt: 

In [ ]:
df= pd.DataFrame.from_dict(age_group)
df.head()


,Gifts and Toys for 1.5 Year Olds (18 Months) - 3 Year Old Toddlers,Gifts and Toys for 4 and 5 Year Old Preschoolers,"Gifts and Toys for 6, 7 and 8 Year Olds","Gifts and Toys for 9, 10, 11 and 12 Year Olds","Gifts and Toys for 13, 14, 15, 16 and 17 Year Old Teens & Young Adults",Toys and Gifts for 18+ Year Old Adults
0,"[3in1 Family House, Mickey & Minnie Birthday T...","[LEGO® Medium Creative Brick Box, LEGO® Minifi...","[Land Rover Classic Defender, Gingerbread Orna...","[Land Rover Classic Defender, London Postcard,...","[Christmas Tree, London, Mosaic Maker, LEGO® H...","[Concorde, Tiny Plants, The Insect Collection,..."
1,"[3+, 2+, 2+, 1½+, 2+, 1½+, 1½+, 1½+, 1½+, 2+, ...","[4-99, 5+, 4-99, 5+, 4+, 4+, 4+, 5+, 5+, 5+, 4...","[8+, 6+, 8+, 7+, 4-99, 8+, 8+, 7+, 7+, 8+, 8+,...","[8+, 9+, 9+, 6+, 7+, 8+, 9+, 8+, 10+, 9+, 7+, ...","[12+, 12+, 10+, 12+, 10+, 10+, 10+, 10+, 10+, ...","[18+, 18+, 18+, 18+, 18+, 18+, 18+, 18+, 18+, ..."
2,"[€139,99, €34,99, €64,99, €14,99, €34,99, €29,...","[€29,99, €3,99, €49,99, €9,99, €9,99, €8,99, €...","[€14,99, €12,99, €29,99, €9,99, €29,99, €12,99...","[€14,99, €14,99, €39,99, €12,99, €24,99, €29,9...","[€44,99, €39,99, €99,99, €49,99, €14,99, €9,99...","[€199,99, €49,99, €79,99, €299,99, €679,99, €6..."
3,[https://www.lego.com/cdn/cs/set/assets/bltcf4...,[https://www.lego.com/cdn/cs/set/assets/blt702...,[https://www.lego.com/cdn/cs/set/assets/blt2c0...,[https://www.lego.com/cdn/cs/set/assets/blt2c0...,[https://www.lego.com/cdn/cs/set/assets/blt254...,[https://www.lego.com/cdn/cs/set/assets/bltdc7...


In [ ]:
age_group_3 = pd.DataFrame.from_dict(age_group['Gifts and Toys for 1.5 Year Olds (18 Months) - 3 Year Old Toddlers']).transpose()
age_group_3.columns= ['title', 'age', 'price', 'image_src']
age_group_3['age_group'] = '1.5-3 Year Old'
age_group_3

,title,age,price,image_src,age_group
0,3in1 Family House,3+,"€139,99",https://www.lego.com/cdn/cs/set/assets/bltcf41...,1.5-3 Year Old
1,Mickey & Minnie Birthday Train,2+,"€34,99",https://www.lego.com/cdn/cs/set/assets/blta199...,1.5-3 Year Old
2,Construction Site,2+,"€64,99",https://www.lego.com/cdn/cs/set/assets/bltaae5...,1.5-3 Year Old
3,LEGO® DUPLO® Green Building Plate,1½+,"€14,99",https://www.lego.com/cdn/cs/set/assets/bltfc1a...,1.5-3 Year Old
4,Lightning McQueen & Mater's Car Wash Fun,2+,"€34,99",https://www.lego.com/cdn/cs/set/assets/bltcf2b...,1.5-3 Year Old
...,...,...,...,...,...
72,Mack at the Race,2+,"€19,99",https://www.lego.com/cdn/cs/set/assets/blt40a5...,1.5-3 Year Old
73,Elsa & Bruni in the Enchanted Forest,2+,"€32,99",https://www.lego.com/cdn/cs/set/assets/blt4a50...,1.5-3 Year Old
74,Frozen Ice Castle,2+,"€49,99",https://www.lego.com/cdn/cs/set/assets/bltc6c2...,1.5-3 Year Old
75,Car Park and Car Wash,2+,"€99,99",https://www.lego.com/cdn/cs/set/assets/bltb78d...,1.5-3 Year Old


In [ ]:
age_group_5 = pd.DataFrame.from_dict(age_group['Gifts and Toys for 4 and 5 Year Old Preschoolers']).transpose()
age_group_5.columns= ['title', 'age', 'price', 'image_src']
age_group_5['age_group'] = '4-5 Year Old'
age_group_5


,title,age,price,image_src,age_group
0,LEGO® Medium Creative Brick Box,4-99,"€29,99",https://www.lego.com/cdn/cs/set/assets/blt7023...,4-5 Year Old
1,LEGO® Minifigures Marvel Series 2,5+,"€3,99",https://www.lego.com/cdn/cs/set/assets/bltd099...,4-5 Year Old
2,LEGO® Large Creative Brick Box,4-99,"€49,99",https://www.lego.com/cdn/cs/set/assets/blt5c96...,4-5 Year Old
3,Twirling Rapunzel,5+,"€9,99",https://www.lego.com/cdn/cs/set/assets/bltc2b7...,4-5 Year Old
4,Spider-Man's Car and Doc Ock,4+,"€9,99",https://www.lego.com/cdn/cs/set/assets/blt007b...,4-5 Year Old
...,...,...,...,...,...
194,Vibrant Creative Brick Box,4+,"€49,99",https://www.lego.com/cdn/cs/set/assets/bltaf65...,4-5 Year Old
195,Team Spidey Web Spinner Headquarters,4+,"€54,99",https://www.lego.com/cdn/cs/set/assets/bltb55a...,4-5 Year Old
196,Zyclops Chase,4+,"€19,99",https://www.lego.com/cdn/cs/set/assets/blt782e...,4-5 Year Old
197,8-Stud Desk Drawer – Red,5+,"€24,99",https://www.lego.com/cdn/cs/set/assets/blta27b...,4-5 Year Old


In [ ]:
age_group_8 = pd.DataFrame.from_dict(age_group['Gifts and Toys for 6, 7 and 8 Year Olds']).transpose()
age_group_8.columns= ['title', 'age', 'price', 'image_src']
age_group_8['age_group'] = '6-8 Year Old'
age_group_8


,title,age,price,image_src,age_group
0,Land Rover Classic Defender,8+,"€14,99",https://www.lego.com/cdn/cs/set/assets/blt2c07...,6-8 Year Old
1,Gingerbread Ornaments,6+,"€12,99",https://www.lego.com/cdn/cs/set/assets/blt0758...,6-8 Year Old
2,Millennium Falcon™ Holiday Diorama,8+,"€29,99",https://www.lego.com/cdn/cs/set/assets/blte9e3...,6-8 Year Old
3,Tuk Tuk,7+,"€9,99",https://www.lego.com/cdn/cs/set/assets/bltc456...,6-8 Year Old
4,LEGO® Medium Creative Brick Box,4-99,"€29,99",https://www.lego.com/cdn/cs/set/assets/blt7023...,6-8 Year Old
...,...,...,...,...,...
923,Notebook with Gel Pen – Yellow,6+,"€16,99",https://www.lego.com/cdn/cs/set/assets/blt1f5c...,6-8 Year Old
924,1x1 Brick NiteLite – White,6+,"€19,99",https://www.lego.com/cdn/cs/set/assets/bltd1eb...,6-8 Year Old
925,LEGO® Brick Lunch Bag – Pink,6+,"€29,99",https://www.lego.com/cdn/cs/set/assets/bltc369...,6-8 Year Old
926,8-Stud Desk Drawer – Black,6+,"€24,99",https://www.lego.com/cdn/cs/set/assets/blt2d7f...,6-8 Year Old


In [ ]:
age_group_12 = pd.DataFrame.from_dict(age_group['Gifts and Toys for 9, 10, 11 and 12 Year Olds']).transpose()
age_group_12.columns= ['title', 'age', 'price', 'image_src']
age_group_12['age_group'] = '9-12 Year Old'
age_group_12


,title,age,price,image_src,age_group
0,Land Rover Classic Defender,8+,"€14,99",https://www.lego.com/cdn/cs/set/assets/blt2c07...,9-12 Year Old
1,London Postcard,9+,"€14,99",https://www.lego.com/cdn/cs/set/assets/blt784d...,9-12 Year Old
2,Santa's Sleigh,9+,"€39,99",https://www.lego.com/cdn/cs/set/assets/bltb7d4...,9-12 Year Old
3,Gingerbread Ornaments,6+,"€12,99",https://www.lego.com/cdn/cs/set/assets/blt0758...,9-12 Year Old
4,Exotic Pink Parrot,7+,"€24,99",https://www.lego.com/cdn/cs/set/assets/blt2910...,9-12 Year Old
...,...,...,...,...,...
914,Notebook with Gel Pen – Violet,6+,"€16,99",https://www.lego.com/cdn/cs/set/assets/blta643...,9-12 Year Old
915,Notebook with Gel Pen – Yellow,6+,"€16,99",https://www.lego.com/cdn/cs/set/assets/blt1f5c...,9-12 Year Old
916,4-Stud Desk Drawer – White,6+,"€15,99",https://www.lego.com/cdn/cs/set/assets/blt8e00...,9-12 Year Old
917,1x1 Brick NiteLite – Blue,6+,"€19,99",https://www.lego.com/cdn/cs/set/assets/blt37aa...,9-12 Year Old


In [ ]:
age_group_17 = pd.DataFrame.from_dict(age_group['Gifts and Toys for 13, 14, 15, 16 and 17 Year Old Teens & Young Adults']).transpose()
age_group_17.columns= ['title', 'age', 'price', 'image_src']
age_group_17['age_group'] = '13-17 Year Old'
age_group_17

,title,age,price,image_src,age_group
0,Christmas Tree,12+,"€44,99",https://www.lego.com/cdn/cs/set/assets/blt2547...,13-17 Year Old
1,London,12+,"€39,99",https://www.lego.com/cdn/cs/set/assets/blt98d7...,13-17 Year Old
2,Mosaic Maker,10+,"€99,99",https://www.lego.com/cdn/cs/set/assets/blt8a83...,13-17 Year Old
3,LEGO® House,12+,"€49,99",https://www.lego.com/cdn/cs/set/assets/blt4ec9...,13-17 Year Old
4,Goofy & Pluto,10+,"€14,99",https://www.lego.com/cdn/cs/set/assets/blt4306...,13-17 Year Old
...,...,...,...,...,...
104,Wedding Bride,10+,"€12,99",https://www.lego.com/cdn/cs/set/assets/blt4ab3...,13-17 Year Old
105,The Razor Crest™,10+,"€139,99",https://www.lego.com/cdn/cs/set/assets/blt3ad5...,13-17 Year Old
106,Robot Inventor,10+,"€359,99",https://www.lego.com/cdn/cs/set/assets/blt7202...,13-17 Year Old
107,Material Handler,10+,"€119,99",https://www.lego.com/cdn/cs/set/assets/blt0bd9...,13-17 Year Old


In [ ]:
age_group_adults = pd.DataFrame.from_dict(age_group['Toys and Gifts for 18+ Year Old Adults']).transpose()
age_group_adults.columns= ['title', 'age', 'price', 'image_src']
age_group_adults['age_group'] = 'Adults'
age_group_adults

,title,age,price,image_src,age_group
0,Concorde,18+,"€199,99",https://www.lego.com/cdn/cs/set/assets/bltdc70...,Adults
1,Tiny Plants,18+,"€49,99",https://www.lego.com/cdn/cs/set/assets/bltb2f8...,Adults
2,The Insect Collection,18+,"€79,99",https://www.lego.com/cdn/cs/set/assets/blte32d...,Adults
3,Natural History Museum,18+,"€299,99",https://www.lego.com/cdn/cs/set/assets/blt08ba...,Adults
4,LEGO® Titanic,18+,"€679,99",https://www.lego.com/cdn/cs/set/assets/bltb946...,Adults
...,...,...,...,...,...
153,Venom,18+,"€69,99",https://www.lego.com/cdn/cs/set/assets/blta53b...,Adults
154,Death Star™ Trash Compactor Diorama,18+,"€89,99",https://www.lego.com/cdn/cs/set/assets/bltd793...,Adults
155,Dagobah™ Jedi™ Training Diorama,18+,"€89,99",https://www.lego.com/cdn/cs/set/assets/bltb94f...,Adults
156,Pickup Truck,18+,"€129,99",https://www.lego.com/cdn/cs/set/assets/blt6a4e...,Adults


In [ ]:
# Concatenate DataFrames vertically
lego_items = pd.concat([age_group_3, age_group_5, age_group_8, age_group_12, age_group_17, age_group_adults], axis=0).reset_index()
lego_items.drop(columns=['index'],inplace=True)
display(lego_items)

,title,age,price,image_src,age_group
0,3in1 Family House,3+,"€139,99",https://www.lego.com/cdn/cs/set/assets/bltcf41...,1.5-3 Year Old
1,Mickey & Minnie Birthday Train,2+,"€34,99",https://www.lego.com/cdn/cs/set/assets/blta199...,1.5-3 Year Old
2,Construction Site,2+,"€64,99",https://www.lego.com/cdn/cs/set/assets/bltaae5...,1.5-3 Year Old
3,LEGO® DUPLO® Green Building Plate,1½+,"€14,99",https://www.lego.com/cdn/cs/set/assets/bltfc1a...,1.5-3 Year Old
4,Lightning McQueen & Mater's Car Wash Fun,2+,"€34,99",https://www.lego.com/cdn/cs/set/assets/bltcf2b...,1.5-3 Year Old
...,...,...,...,...,...
2385,Venom,18+,"€69,99",https://www.lego.com/cdn/cs/set/assets/blta53b...,Adults
2386,Death Star™ Trash Compactor Diorama,18+,"€89,99",https://www.lego.com/cdn/cs/set/assets/bltd793...,Adults
2387,Dagobah™ Jedi™ Training Diorama,18+,"€89,99",https://www.lego.com/cdn/cs/set/assets/bltb94f...,Adults
2388,Pickup Truck,18+,"€129,99",https://www.lego.com/cdn/cs/set/assets/blt6a4e...,Adults


In [ ]:
lego_items.to_csv('lego_catalog.csv', index=False)


In [2]:
!pip freeze

aiobotocore @ file:///C:/b/abs_74o47svlua/croot/aiobotocore_1680004300264/work
aiofiles @ file:///C:/b/abs_9ex6mi6b56/croot/aiofiles_1683773603390/work
aiohttp @ file:///C:/ci_311/aiohttp_1676432932774/work
aioitertools @ file:///tmp/build/80754af9/aioitertools_1607109665762/work
aiosignal @ file:///tmp/build/80754af9/aiosignal_1637843061372/work
aiosqlite @ file:///C:/b/abs_9djc_0pyi3/croot/aiosqlite_1683773915844/work
alabaster @ file:///home/ktietz/src/ci/alabaster_1611921544520/work
altair==5.2.0
anaconda-anon-usage @ file:///C:/b/abs_4cqo3nmgh0/croot/anaconda-anon-usage_1694468596737/work
anaconda-catalogs @ file:///C:/b/abs_8btyy0o8s8/croot/anaconda-catalogs_1685727315626/work
anaconda-client==1.12.0
anaconda-navigator==2.4.3
anaconda-project @ file:///C:/ci_311/anaconda-project_1676458365912/work
anyio @ file:///C:/ci_311/anyio_1676425491996/work/dist
appdirs==1.4.4
argon2-cffi @ file:///opt/conda/conda-bld/argon2-cffi_1645000214183/work
argon2-cffi-bindings @ file:///C:/ci_311/